James Gardner, 2022 

want to analyse science case/s for CE only:

CE-N 40km with CE-S 40km or 20km

if done, then look at CE-S with one ET detector

In [ ]:
from benchmarking import *

### Replicating Borhanian and Sathya 2022 injections and detection rates, then for CE only 

In [ ]:
# structure: network, injection loop (inj, benchmark, save data), plot snr histogram, ...
# ... calculate efficiency, calculate detection rate

# try for this network, then CE only (refer to App E for CE discussion), compare to other five in Section 2a?
# --- HLVKI+ ---
network_spec = ['A+_H', 'A+_L', 'V+_V', 'K+_K', 'A+_I']
locs = [x[-1] for x in network_spec]
net = network.Network(network_spec)

# --- BNS ---
# waveform, LAL list: https://lscsoft.docs.ligo.org/lalsuite/lalsimulation/group___l_a_l_sim_inspiral__h.html
# wf_model_name = 'lal_bns'
# wf_other_var_dic = dict(approximant='IMRPhenomD_NRTidalv2') # for tidal, see https://arxiv.org/abs/1905.06011
# to-do: fix "TypeError: hfpc() missing 2 required positional arguments: 'lam_t' and 'delta_lam_t'"
# --> calculate tidal parameters from sampled m1, m2 in injections.py? requires Love number and radii (i.e. choose an EoS)
wf_model_name, wf_other_var_dic = 'tf2_tidal', None # to-do: stop using this once tidal params found
net.set_wf_vars(wf_model_name=wf_model_name, wf_other_var_dic=wf_other_var_dic)
# injection settings - source
mass_dict = dict(dist='gaussian', mean=1.35, sigma=0.15, mmin=1, mmax=2)
spin_dict = dict(geom='cartesian', dim=1, chi_lo=-0.05, chi_hi=0.05)
# zmin, zmax, seed (use same seeds to replicate results)
redshift_bins = ((0, 0.5, 7669), (0.5, 1, 3103), (1, 2, 4431), (2, 4, 5526), (4, 10, 7035), (10, 50, 2785))
coeff_fisco = 4 # fmax = 4*fisco for BNS, 8*fisco for BBH

base_params = {
    'tc':    0,
    'phic':  0,
    'gmst0': 0, # assume zero given B2021
    'lam_t': 800, # combined dimensionless tidal deformability, 800 for GW170817, to-do: figure out
    'delta_lam_t': 0, # assuming zero but can be calculated if m1, m2, Love number, and EoS (i.e. radii) known
}

# derivative settings
# assign with respect to which parameters to take derivatives for the FIM
deriv_symbs_string = 'Mc eta DL tc phic iota ra dec psi'
# assign which parameters to convert to log or cos versions for differentiation
conv_cos = ('dec', 'iota')
conv_log = ('Mc', 'DL', 'lam_t')
numerical_deriv_settings = dict(step=1e-9, method='central', order=2, n=1) # default

# network settings: whether to include Earth's rotation and individual detector calculations
use_rot = 1
only_net = 1

# injection settings - other: number of injections per redshift bin (over 6 bins)
num_injs = 100 # start with 10, then build to 1e6 (how did they compute 1e6 with numerical derivs?)
redshifted = 1 # whether sample masses already redshifted wrt z
if wf_other_var_dic is not None:
    file_tag = f'NET_{net.label}_WF_{wf_model_name}_{wf_other_var_dic["approximant"]}_NUM-INJS_{num_injs}'
    human_file_tag = f'network: {net.label.replace("..", ", ")}\nwaveform: {wf_model_name} with {wf_other_var_dic["approximant"]}\nnumber of injections per bin: {num_injs}'
else:
    file_tag = f'NET_{net.label}_WF_{wf_model_name}_NUM-INJS_{num_injs}'
    human_file_tag = f'network: {net.label.replace("..", ", ")}\nwaveform: {wf_model_name}\nnumber of injections per bin: {num_injs}'    
    
# if tf2 or tf2_tidal used
generate_symbolic_derivatives(wf_model_name, wf_other_var_dic, deriv_symbs_string, locs, use_rot)

In [ ]:
# injection and benchmarking
# concatenate injection data from different bins
inj_data = np.empty((len(redshift_bins)*num_injs, 14))
for i, (zmin, zmax, seed) in enumerate(redshift_bins):
    cosmo_dict = dict(sampler='uniform', zmin=zmin, zmax=zmax)
    # transposed array to get [[Mc0, eta0, ..., z0], [Mc1, eta1, ..., z1], ...]
    # [Mc, eta, chi1x, chi1y, chi1z, chi2x, chi2y, chi2z, DL, iota, ra, dec, psi, z]    
    inj_data[i*num_injs:(i+1)*num_injs] = np.array(injections.injections_CBC_params_redshift(cosmo_dict, mass_dict, spin_dict, redshifted, num_injs=num_injs, seed=seed)).transpose()

def calculate_benchmark_from_injection(inj):
    """given a 14-array of [Mc, eta, chi1x, chi1y, chi1z, chi2x, chi2y, chi2z, DL, iota, ra, dec, psi, z],
    returns a 7-tuple of the
    * redshift z,
    * integrated snr,
    * fractional Mc and DL and absolute eta and iota errors,
    * 90% sky area.
    sigma_log(Mc) = sigma_Mc/Mc is fractional error in Mc and similarly for DL, sigma_eta is absolute,
    while |sigma_cos(iota)| = |sigma_iota*sin(iota)| --> error in iota requires rescaling from output"""
    varied_keys = ['Mc', 'eta', 'chi1x', 'chi1y', 'chi1z', 'chi2x', 'chi2y', 'chi2z', 'DL', 'iota', 'ra', 'dec', 'psi', 'z']
    varied_params = dict(zip(varied_keys, inj))
    z = varied_params.pop('z')
    Mc, eta, iota = varied_params['Mc'], varied_params['eta'], varied_params['iota']
    
    Mtot = Mc/eta**0.6
    #fisco = (6**1.5*PI*Mtot)**-1 # missing some number of Msun, c=1, G=1 factors
    fisco = 4.4/Mtot*1e3 # Hz # from https://arxiv.org/pdf/2011.05145.pdf
    fmin, fmax, df = 5., float(min(coeff_fisco*fisco, 1024)), 10 # df=1/16 (fine) 
    f = np.arange(fmin, fmax, df)

    # net_copy is automatically deleted once out of scope (is copying necessary with Pool()?)
    net_copy = deepcopy(net)
    inj_params = dict(**base_params, **varied_params)
    net_copy.set_net_vars(f=f, inj_params=inj_params, deriv_symbs_string=deriv_symbs_string,
                          conv_cos=conv_cos, conv_log=conv_log, use_rot=use_rot)

    basic_network_benchmarking(net_copy, numerical_over_symbolic_derivs=False, only_net=only_net,
                               numerical_deriv_settings=numerical_deriv_settings, hide_prints=True)

    if net_copy.wc_fisher:
        # convert sigma_cos(iota) into sigma_iota
        abs_err_iota = abs(net_copy.errs['cos_iota']/np.sin(iota))
        return (z, net_copy.snr, net_copy.errs['log_Mc'], net_copy.errs['log_DL'], net_copy.errs['eta'],
                abs_err_iota, net_copy.errs['sky_area_90'])
    else:
        # to-do: check if CE only is still ill-conditioned
        return (z, *np.full(6, np.nan))

In [ ]:
# calculate results: z, snr, errs (logMc, logDL, eta, iota), sky area
# p_umap is unordered in redshift for greater speed (check)
results = np.array(p_umap(calculate_benchmark_from_injection, inj_data, num_cpus=os.cpu_count()-1))
# filter out NaNs
results = without_rows_w_nan(results)
if len(results) == 0:
    print('All values are NaN, FIM is ill-conditioned.')
np.save(f'data_B&S2022_replication/results_{file_tag}.npy', results)

In [ ]:
results = np.load(f'data_B&S2022_replication/results_{file_tag}.npy')

In [ ]:
# count efficiency over sources in (z, z+Delta_z) for z=0.01 to 50
zmin_plot, zmax_plot, num_zbins_fine = 1e-2, 50, 20
redshift_bins_fine = list(zip(np.geomspace(zmin_plot, zmax_plot, num_zbins_fine)[:-1],
                              np.geomspace(zmin_plot, zmax_plot, num_zbins_fine)[1:])) # redshift_bins are too wide
zavg_efflo_effhi = np.empty((len(redshift_bins_fine), 3))
for i, (zmin, zmax) in enumerate(redshift_bins_fine):
    z_snr_in_bin = results[:,0:2][np.logical_and(zmin < results[:,0], results[:,0] < zmax)]
    zavg_efflo_effhi[i,0] = np.mean(z_snr_in_bin[:,0]) # maybe should be gmean because z plotted logarithmically?
    zavg_efflo_effhi[i,1] = np.mean(z_snr_in_bin[:,1] > SNR_THRESHOLD_LO)
    zavg_efflo_effhi[i,2] = np.mean(z_snr_in_bin[:,1] > SNR_THRESHOLD_HI)

# plotting
plt.rcParams.update({'font.size': 14})
fig, axs = plt.subplots(2, 1, sharex=True, figsize=(6, 8), gridspec_kw={'wspace':0, 'hspace':0.05})

# snr vs redshift
# use integrated SNR rho from standard benchmarking, not sure if B&S2022 use matched filter
axs[0].loglog(results[:,0], results[:,1], '.')
axs[0].axhspan(0, SNR_THRESHOLD_LO, alpha=0.5, color='lightgrey')
axs[0].axhspan(0, SNR_THRESHOLD_HI, alpha=0.25, color='lightgrey')
axs[0].set_ylabel(r'integrated SNR, $\rho$')
axs[0].set_title(human_file_tag, fontsize=14)

# efficiency vs redshift
axs[1].axhline(0, color='grey', linewidth=0.5)
axs[1].axhline(1, color='grey', linewidth=0.5)
axs[1].plot(zavg_efflo_effhi[:,0], zavg_efflo_effhi[:,1], 'o', color='darkred', label=fr'$\rho$ > {SNR_THRESHOLD_LO}')
axs[1].plot(zavg_efflo_effhi[:,0], zavg_efflo_effhi[:,2], 's', color='darkred', label=fr'$\rho$ > {SNR_THRESHOLD_HI}')
axs[1].legend()
axs[1].set_ylim((0-0.05, 1+0.05))
axs[1].set_ylabel(r'detection efficiency, $\varepsilon$')
axs[1].set_xscale('log')
axs[1].set_xlim((1e-2, 50))
axs[1].xaxis.set_minor_locator(plt.LogLocator(base=10.0, subs=0.1*np.arange(1, 10), numticks=10))
axs[1].xaxis.set_minor_formatter(plt.NullFormatter())
axs[1].set_xlabel('redshift, z')

fig.savefig(f'snr_vs_redshift_{file_tag}.pdf', bbox_inches='tight')
plt.show(fig)
plt.close(fig)

In [ ]:
# to-do: decouple (z, z+dz) bins to plot efficiency from injection redshift bins (the latter are too wide)

In [ ]:
# to-do: increase sampling below 4e-2, but App A uses (0, 0.5, seed)?

In [ ]:
# detection efficiency
det_eff = lambda z, snr_threshold : ?

# detection rate integrand
det_rate_integrand = lambda z, snr_threshold : det_eff(z, snr_threshold)*injections.bns_md_merger_rate(z)/(1+z)

# detection rate
det_rate = lambda z0, snr_threshold : scipy.integrate.quad(lambda z : det_rate_integrand(z, snr_threshold), 0, z0)

import scipy.integrate

In [ ]:
# to-do: pass injection data to benchmarking, then calculate a detection rate!
# injections.bns_md_merger_rate(0) # from z=0 outwards
plt.rcParams.update({'font.size': 14})
fig, ax = plt.subplots()
zarr = np.logspace(np.log10(1e-2), np.log10(50), 100)
ax.plot(zarr, list(map(injections.bns_md_merger_rate, zarr)))
ax.set_xscale('log')
ax.set_xlim((1e-2, 50))
ax.set(xlabel='redshift, z', ylabel=r'merger rate / $\mathrm{Gpc}^{-3} \mathrm{yr}^{-1}$') # check units?
plt.show(fig)
plt.close(fig)

In [ ]:
# to-do: fit three-parameter sigmoids to efficiency curves vs redshift

In [ ]:
# to-do: study BBH science case as well

In [ ]:
"""In fact, we see that the three generations (A+, Voyager, and NG) are qualitatively different
with respect to every metric used in this study."""